<a href="https://colab.research.google.com/github/SubashKhatiwada/RSA-vs-ECC/blob/main/ECC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
import time

# Generate Elliptic Equation 

In [ ]:
def generate_b(p:int, a:int, G:list):
  xg = G[0]
  yg = G[1]
  xg3 = pow(xg,3)
  axg = (a * xg)
  xg3plusaxg = xg3 + axg
  yg2 = pow(yg,2)
  b = (yg2 - xg3plusaxg) % p
  return b

p= 503
a = 4
G = (283,315)
b = 1

print("a = ",a)
print("b = ",b)
print("p = ",p)
print("G = ",G)

a =  4
b =  1
p =  503
G =  (283, 315)


#Global Parameters Generation 

In [ ]:
a =  4
b =  1
p =  503
G =  [283, 315]

# Find Total Points in Ep(a,b)

In [ ]:
# #Total points on elliptic curve under modulo p# 
# def total_points(a:int, b:int, p:int):
#   points_set = []
#   for  x in range(0,p-1):
#     y_G = pow((pow(x,3)+a*x+b),1,p)
#     for y in range(0,p-1):
#       if pow(y,2,p) == y_G:
#         points_set.append([x,y])
#         break
#   return points_set

15


# Elliptic Curve Operations

In [ ]:
# Extended Euclidean Algorithm for calculating Multiplicative Inverse#
def euclides(inverso, modulo):
    modulo0 = modulo
    x0, x1, y0, y1 = 1, 0, 0, 1
    while modulo != 0:
        q, inverso, modulo = inverso // modulo, modulo, inverso % modulo
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1    
    if x0 < 0:
        x0 += modulo0
    return x0

def curve_operation(P:list,Q:list):
  if P == Q:
    R = point_doubling(P)
  elif P[0] == Q[0] & P[1] == -Q[1] % p:
    R = 0
  else: 
    R = point_addition(P,Q)
  return R

def point_addition(P:list,Q:list):
  x1, y1, x2, y2 = P[0], P[1], Q[0], Q[1] 
  numerator = pow((y2-y1),1,p)
  denominator = pow((x2-x1),1,p)
  inverse_denominator = euclides(denominator,p)
  lamda = pow(numerator * inverse_denominator,1,p)
  x3 = pow(pow(lamda,2,p)-x1-x2,1,p)
  y3 = pow(lamda*(x1-x3)-y1,1,p)
  R = [x3,y3]
  return R

def point_doubling(P:list):
  x1, y1 = P[0], P[1]
  numerator = pow(3*pow(x1,2)+a,1,p)
  denominator = pow(2*y1,1,p)
  inverse_denominator = euclides(denominator,p)
  lamda = pow(numerator * inverse_denominator,1,p)
  x3 = pow(pow(lamda,2)-2*x1,1,p)
  y3 = pow(lamda*(x1-x3)-y1,1,p)
  R = [x3,y3]
  return R

def point_multiplication(k:int,P:list):
  R=P
  for i in range(2,k+1):
    R = curve_operation(R,P)
    if R==0:
      break
  return R

# print(point_multiplication(1,[283,315]))
  

# Key Generation for Alice

In [ ]:
alice_private_key = 121
alice_public_key = point_multiplication(alice_private_key,G)
print(alice_public_key)

[453, 78]


# Key Generation for Bob

In [ ]:
bob_private_key = 101
bob_public_key = point_multiplication(bob_private_key,G)
print(bob_public_key)

[363, 109]


# Shared Secret Key Generation

In [ ]:
alice_secret_key = point_multiplication(alice_private_key, bob_public_key)
bob_secret_key = point_multiplication(bob_private_key, alice_public_key)
print(alice_secret_key)
print(bob_secret_key)

[45, 303]
[45, 303]


# Message Encoder and Decoder

In [ ]:
def generate_table(G:list):
  table = []
  for i in range(0,127):
    point = point_multiplication(i,G)
    table.append([i,point])
  return table
mapping_table = generate_table(G)

def encoder(text:str):
  ascii_list = [ord(c) for c in text]
  encoded_points = []
  for i in range(0,len(ascii_list)):
    encoded_point = mapping_table[ascii_list[i]][1]
    encoded_points.append(encoded_point)
  return encoded_points

def decoder(encoded_points:list):
  points_list = []
  ascii_list = []
  for j in range(127):
    value = mapping_table[j][1]
    points_list.append(value)
  indices = []
  for i in range(len(encoded_points)):
    index = points_list.index(encoded_points[i])
    ascii_list.append(index)
  message = ''.join(chr(i) for i in ascii_list)
  return(message)

# ECC Message Encryption by Alice




In [ ]:
def encryption(message:str):
  Pm = encoder(message)
  Pm_plus_kPB_list = []
  k = 41
  print("k = ", k)
  kG = point_multiplication(k,G)
  kPb = point_multiplication(k,bob_public_key)
  print("kPUb = ", kPb)
  for i in range(0,len(Pm)):
    Pm_plus_kPB = point_addition(Pm[i],kPb)
    Pm_plus_kPB_list.append(Pm_plus_kPB)
  cipher = [kG,Pm_plus_kPB_list]
  print("C1 = ", kG)
  print("C2=",Pm_plus_kPB_list)
  print("cipher = ", cipher)
  return(cipher)


# ECC Chipher Decryption by Bob 

In [ ]:
def decryption(cipher:list):
  kG = cipher[0]
  print("C1(kG) = ", kG)
  print("bob_private_key = ", bob_private_key)
  Pm_plus_kPB_list = cipher[1]
  nBkG = point_multiplication(bob_private_key,kG)
  print("nBkG = ",nBkG)
  negative_nBkG = [nBkG[0], pow(-nBkG[1],1,p)]
  print("-nBkG = ", negative_nBkG)
  Pm = []
  for i in range(0,len(Pm_plus_kPB_list)):
    decry_p = curve_operation(Pm_plus_kPB_list[i],negative_nBkG)
    Pm.append(decry_p)
  print("Encoded message =",Pm)
  message = decoder(Pm)
  print("Message=", message)

  return message


# Implementation

In [ ]:
m = "H"

enc_starttime = time.time()
Cm = encryption(m)
enc_endtime = time.time()
dec_starttime = time.time()
Pm = decryption(Cm)
dec_endtime = time.time()
print("Encryption Time = ", enc_endtime - enc_starttime)
print("Decryption Time = ", dec_endtime - dec_starttime)



k =  41
kPUb =  [185, 213]
C1 =  [143, 474]
C2= [[308, 154]]
cipher =  [[143, 474], [[308, 154]]]
C1(kG) =  [143, 474]
bob_private_key =  101
nBkG =  [185, 213]
-nBkG =  [185, 290]
Encoded message = [[265, 52]]
Message= H
Encryption Time =  0.006996870040893555
Decryption Time =  0.002674102783203125
